# Seq2Seq AdaLoRa for C++

This is the main notebook for fine-tuning *codet5p-220m-bimodal seq2seq* using AdaLoRa PEFT method on C++ programming language dataset.

## Clone the repository

In [1]:
!git clone https://github.com/leiluk1/CodeSearcher.git && cd CodeSearcher/ && git checkout dev/embeddings

Cloning into 'CodeSearcher'...
remote: Enumerating objects: 421, done.
remote: Counting objects: 100% (61/61), done.
remote: Compressing objects: 100% (58/58), done.
remote: Total 421 (delta 5), reused 18 (delta 2), pack-reused 360
Receiving objects: 100% (421/421), 40.58 MiB | 6.91 MiB/s, done.
Resolving deltas: 100% (220/220), done.
Branch 'dev/embeddings' set up to track remote branch 'dev/embeddings' from 'origin'.
Switched to a new branch 'dev/embeddings'


## Set up the required dependencies

In [ ]:
!pip install dataprep gdown py7zr transformers peft evaluate rouge_score fire loguru --quiet

In [3]:
# import wandb
# wandb.login(key='')

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [4]:
!mkdir -p /kaggle/output/CodeSearcher/output
!mkdir -p CodeSearcher/data/raw
!gdown 1tZfsYQgWmc2gG340ru5VbrZ5aLIZ41_6
!unzip -q -d /kaggle/working/CodeSearcher/data/raw ./XLCoST_data.zip

Downloading...
From (uriginal): https://drive.google.com/uc?id=1tZfsYQgWmc2gG340ru5VbrZ5aLIZ41_6
From (redirected): https://drive.google.com/uc?id=1tZfsYQgWmc2gG340ru5VbrZ5aLIZ41_6&confirm=t&uuid=effffb65-cc9b-48da-8b2b-c83406c4dd24
To: /kaggle/working/XLCoST_data.zip
100%|█████████████████████████████████████████| 298M/298M [00:02<00:00, 113MB/s]


## Perform fine-tuning 

For more details, please check the code in `src/models/train.py` in our repository.

In [5]:
!export DATASETS_VERBOSITY=error
!cd CodeSearcher/ && export PYTHONPATH=. && python src/models/train.py seq2seq adalora \
    --output_dir="output" \
    --epochs=10 \
    --language="C++" \
    --train_batch_size=16

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
A new version of the following files was downloaded from https://huggingface.co/Salesforce/codet5p-220m-bimodal:
- configuration_codet5p_bimodal.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/Salesforce/codet5p-220m-bimodal:
- modeling_codet5p_bimodal.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
2023-11-27 10:15:13.293 | INFO     | __ma

## Save checkpoint

In [6]:
!zip -r seq2seq_adalora_c.zip CodeSearcher/output

  adding: CodeSearcher/output/ (stored 0%)
  adding: CodeSearcher/output/checkpoint-3500/ (stored 0%)
  adding: CodeSearcher/output/checkpoint-3500/tokenizer.json (deflated 72%)
  adding: CodeSearcher/output/checkpoint-3500/adapter_model.safetensors (deflated 8%)
  adding: CodeSearcher/output/checkpoint-3500/special_tokens_map.json (deflated 82%)
  adding: CodeSearcher/output/checkpoint-3500/scheduler.pt (deflated 49%)
  adding: CodeSearcher/output/checkpoint-3500/vocab.json (deflated 59%)
  adding: CodeSearcher/output/checkpoint-3500/added_tokens.json (deflated 37%)
  adding: CodeSearcher/output/checkpoint-3500/optimizer.pt (deflated 8%)
  adding: CodeSearcher/output/checkpoint-3500/README.md (deflated 65%)
  adding: CodeSearcher/output/checkpoint-3500/adapter_config.json (deflated 50%)
  adding: CodeSearcher/output/checkpoint-3500/merges.txt (deflated 54%)
  adding: CodeSearcher/output/checkpoint-3500/trainer_state.json (deflated 76%)
  adding: CodeSearcher/output/checkpoint-3500/tok

In [10]:
from IPython.display import FileLink

FileLink('seq2seq_adalora_c.zip')

/kaggle/working/seq2seq_adalora_c.zip

## Evaluation

In this step, test the model using *Mean Reciprocal Rank (MRR)* as the evaluation metric. 

For more details, please refer to `src/models/evaluation.py` in the repository.

In [7]:
!cd CodeSearcher/ && export PYTHONPATH=. && python src/models/evaluation.py peft \
    --tuned_ckpt_path="output/best_ckpt" \
    --language="C++" 

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
2023-11-27 19:18:23.949 | INFO     | src.datasets.XLCoST.make_dataset:_load_search_dataframe:84 - Loading dataframe from data/raw/XLCoST_data/retrieval/nl2code_search/snippet_level/C++/train.jsonl
/kaggle/working/CodeSearcher/src/datasets/XLCoST/make_dataset.py:93: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe_trunc['code_tokens'] = dataframe_trunc['code_tokens'].apply(_code_

# Results:

- Test MRR for C#: **0.11528267**;
- Epochs: **10**;
- Trainable params: 1,327,968 || all params: 224,410,794 || trainable%: 0.5917576317652528.

You can also check the plots for validation results via link to [wanb report](https://api.wandb.ai/links/ley-khaertdinova/13lvn64p).
